
# DATA QUALITY (Bibtex e csv)
- Arquivo texto em formato BIBTEX
- Objetivo: Data quality de input diversos(csv, bibtex e APIs)

In [1]:
import os
import pandas as pd
import re
import hashlib
import yaml
import json
import csv
import requests
from numpy import nan
from pybtex.database.input import bibtex
from pybtex.database import BibliographyData, Entry
from unicodedata import normalize

from flask import jsonify, Flask, render_template, request
import werkzeug

pd.set_option("display.max_columns", 50)
pd.set_option("display.max_rows", 100)


##### Criação de funções

In [2]:
######################## LISTAGEM DE ARQUIVOS
def arquivos(path, fendwith = ''):
    if fendwith not in '':
        caminhos = {nome : os.path.join(path, nome) for nome in os.listdir(path) if nome.endswith(fendwith)}
    else:
        caminhos = {nome : os.path.join(path, nome) for nome in os.listdir(path)}
    return caminhos

######################## LISTAGEM DE AUTORES
def author_names(author):
    try:
        return author.persons['author'][0:]
    except:
        return {'author': [(u'none, none')]}

######################## CONCATENA AUTORES
def join_names(person):
    try:
        return person.last_names[0] + ', ' + person.first_names[0]
    except:
        return []

######################## OUTPUT POR TIPO DE EXTENSAO(JSON, YAML E CSV)
def type_output_file(o_data, f_type, filename, path):

    path_complete = path + filename + '.' + f_type

    if f_type == 'yaml':
        with open(path_complete, 'w') as nfile:
            yaml.dump(o_data.to_dict(orient='records'), nfile)

    if f_type == 'json':
        with open(path_complete, 'w') as nfile:
            result = o_data.to_json(orient="records")
            parsed = json.loads(result)
            json.dump(parsed, nfile, indent=4)  

    if f_type == 'csv':
        o_data.to_csv(path_complete, sep='§')

######################## CARGA DO ARQUIVO DE CONFIGURACAO
def load_config(n_file):
    with open(n_file, 'r') as config_file:
        return yaml.load(config_file, Loader=yaml.SafeLoader)

## Inicia importanção dos arquivos

### BIBTEX
- ref bibtex doc: http://paginapessoal.utfpr.edu.br/jamhour/publicacoes/arquivos/00_Compilado_JabRef_dez2015.pdf

In [3]:
source_path = "C:\\Users\\victo\\PycharmProjects\\BibtexDataQuality\\source\\"


lst_files = arquivos(source_path, '.bib')
parser = bibtex.Parser()
# file = parser.parse_file(lst_files['ACM.bib'])
dict_file_fields = {}
list_file_fields = []

#verifica a estrutura de campos para cada tipo de publicação
#cria um dicionario usando como chave a fonte e o tipo de publicação
for f in lst_files:
    parser = bibtex.Parser()
    file = parser.parse_file(lst_files[f])
    f_name = f.replace('.bib','')
    for i in file.entries.values():
        
        for a in sorted(i.fields.keys()):
           
            if a not in list_file_fields:
                list_file_fields.append(a)
list_file_fields.remove('ISSN')

##### importanção e tratamento

In [4]:
lst_files = arquivos(source_path)
parser = bibtex.Parser()
# file = parser.parse_file(lst_files['IEEE.bib'])
dict_bib = {}

for f in lst_files:
    
    ############# IEEE
    if 'IEEE' in f :
        parser = bibtex.Parser()
        file = parser.parse_file(lst_files[f])

        for i in file.entries.values():
            key = i.key
            dict_bib[key] = {i.fields.get('fields', fields) : i.fields.get(fields, '')\
                            for fields in list_file_fields}
            
            dict_bib[key]['type_publication'] = i.type
            dict_bib[key]['author'] = [join_names(person) for person in i.persons.get('author', '')]

            dict_bib[key].update({'keywords' : dict_bib[key]['keywords'].split(';')})
    
    ############## ACM
    if 'acm' in f:
        parser = bibtex.Parser()
        file = parser.parse_file(lst_files[f])

        for i in file.entries.values():
            key = i.key
            dict_bib[key] = {i.fields.get('fields', fields) : i.fields.get(fields, '')\
                            for fields in list_file_fields}
            
            dict_bib[key]['type_publication'] = i.type
            dict_bib[key]['author'] = [join_names(person) for person in i.persons.get('author', '')]

            dict_bib[key].update({'keywords' : dict_bib[key].get('keywords', '').split(';')})

            if i.type == 'inbook':
                dict_bib[key].update({'doi' : dict_bib[key].get('url', '').replace('https://doi.org/', '')})

    ############## Science Direct
    if 'ScienceDirect' in f:
        parser = bibtex.Parser()
        file = parser.parse_file(lst_files[f])

        for i in file.entries.values():
            key = i.key
            dict_bib[key] = {i.fields.get('fields', fields) : i.fields.get(fields, '')\
                        for fields in list_file_fields}
            
            dict_bib[key]['type_publication'] = i.type
            dict_bib[key]['author'] = [join_names(person) for person in i.persons.get('author', '')]
            dict_bib[key].update({'keywords' : dict_bib[key]['keywords'].split(';')})
            
            dict_bib[key].update({'doi' : i.fields['doi'].replace('https://doi.org/', '')})

df_bib = pd.DataFrame(data=dict_bib.values())

In [5]:
print(dict_bib['10.1145/3502771.3502781'])

{'abstract': 'Cyber-physical systems (CPS) have been developed in many industrial sectors and application domains in which the quality requirements of data acquired are a common factor. Data quality in CPS can deteriorate because of several factors such as sensor faults and failures due to operating in harsh and uncertain environments. How can software engineering and artificial intelligence (AI) help manage and tame data quality issues in CPS? This is the question we aimed to investigate in the SEA4DQ workshop. Emerging trends in software engineering need to take data quality management seriously as CPS are increasingly datacentric in their approach to acquiring and processing data along the edge-fog-cloud continuum. This workshop provided researchers and practitioners a forum for exchanging ideas, experiences, understanding of the problems, visions for the future, and promising solutions to the problems in data quality in CPS. Examples of topics include software/hardware architecture

#### Output BIBTEX

In [6]:
config = load_config('config.yaml')        

output_path = config['output_path'][0]
f_name = config['file_name'][0]
file_ext = config['output_ext'][0]

type_output_file(df_bib, file_ext, f_name, output_path)

### CSV SCIMAGO | JSR

##### Criando Funções

In [7]:
source_excel = "C:\\Users\\victo\\PycharmProjects\\BibtexDataQuality\\source\\excel"
lst_excel = arquivos(source_excel)

######################## REMOVE CARACTERES ESPECIAIS
def rmscaract(text):
    result = [re.sub(r"[^a-zA-Z0-9]","", normalize('NFKD', words).encode('ASCII','ignore').decode('ASCII').lower()) for words in text.fillna('')]
    return result

######################## CRIA UM HASH A PARTIR DE UMA STRING
def stringhash(instr):
    hashnum = [hashlib.md5(ikeys.encode()).hexdigest() for ikeys in instr]
    return hashnum

######################## CRIA HASH JÁ COM STRING TRATADA
def hashkey(strkeys):
    stringadjs = rmscaract(strkeys)
    hashresult = [hashlib.md5(ikeys.encode()).hexdigest() for ikeys in stringadjs]
    return hashresult

######################## FUNÇÃO PARA FILTRAGEM
def filter_exp(df):
    a = config['filter_field'][0] + config['search_operator'][0] + "'" + config['search_value'][0] + "'"
    return df.query(a)

##### importanção e tratamento

###### JCS

In [8]:
df_jcs = pd.read_csv(lst_excel['jcs_2020.csv'], delimiter=';').drop_duplicates(subset='Full Journal Title')
cols = ['Rank', 'Full Journal Title','Total Cites', 'Journal Impact Factor', 'Eigenfactor Score']

df_jcs = df_jcs[cols]
df_jcs['hashid'] =  hashkey(df_jcs['Full Journal Title'])
df_jcs = df_jcs.rename(columns={'Journal Impact Factor' : 'jcr_value'})

# df_jcs['hashid'].count()
# df_jcs

###### SCIMAGO

In [9]:
df_scimago = pd.read_csv(lst_excel['scimagojr 2020.csv'], delimiter=';').drop_duplicates()

#cria coluna com hash do title para cruzamento com bibtex 
df_scimago['hashid'] =  hashkey(df_scimago['Title'])

#tratamento coluna issn: splita o valor em outras linhas e dropa coluna antiga
df_scimago['issnkey'] =  df_scimago.Issn.apply(lambda x : x.split(','))
df_scimago = df_scimago.drop(columns=['Issn'])
df_scimago = df_scimago.explode('issnkey')

#renomendo coluna SJR para scimago_value
df_scimago = df_scimago.rename(columns={'SJR' : 'scimago_value'})

# df_scimago 

C:\Users\victo\AppData\Local\Temp\ipykernel_16736\1289106711.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_scimago = pd.read_csv(lst_excel['scimagojr 2020.csv'], delimiter=';').drop_duplicates()


###### JUNÇÃO JCS | SCIMAGO

In [109]:
# Junção Scimago | JCS
dfjournalrank = df_scimago.merge(df_jcs, left_on=['hashid'], right_on=['hashid'],how='outer')

# Para os casos onde o campo Title está nulo preenche com o Full Journal Title
dfjournalrank['Title'] = (dfjournalrank.Title.combine_first(dfjournalrank['Full Journal Title'])).str.lower()
print(dfjournalrank[['Full Journal Title','Title']].count())

# Dropa as colunas duplicadas
dfjournalrank = dfjournalrank.drop(columns=['Rank_x', 'Rank_y', 'Full Journal Title'])
dfjournalrank = dfjournalrank.rename(columns={'Title' : 'JournalTitle'})
# dfjournalrank.count()

Full Journal Title    19525
Title                 50224
dtype: int64


## JOIN BIBTEX | CSV SCIMAGO | CSV JCS

In [11]:
df_bib = pd.DataFrame(data=dict_bib.values())
df_bib = df_bib.replace(r'^\s*$', nan, regex=True)

df_bib['issnkey'] = rmscaract(df_bib['issn'])
df_bib['title'] = rmscaract(df_bib['title'])
df_bib['hashid'] = hashkey(df_bib['title'])

In [12]:
cols = ['issnkey', 'isbn', 'journal', 'publisher', 'title', 'booktitle', 'doi', 'author', 'keywords', 'abstract', 'year', 'type_publication', 'jcr_value', 'scimago_value']

# Junção BIBTEX | SCIMAGO | JSC e filtra as colunas principais
df_join_bibtex_csv = df_bib.merge(dfjournalrank, left_on = 'issnkey', right_on = 'issnkey', how = 'left' )
df_join_bibtex_csv = df_join_bibtex_csv[cols]

In [13]:
#Carrega os parametros do arquivo de configuração
config = load_config('config.yaml')        

# Seta os paramtros nas variaveis
output_path = config['output_path'][0]
f_name = config['file_name'][0]
file_ext = config['output_ext'][0]

# Utiliza funcao de filtragem
df_filter = filter_exp(df_join_bibtex_csv)

# Utiliza funcao para o output do arquivo de acordo com os parametros do arquivo de configuração yaml
type_output_file(df_filter, file_ext, f_name, output_path)

## API

In [72]:
def get_data(n_api, accesskey, max_pages = 1, c_filter = None):

    lst = []

    if 'IEEE' in n_api:
        accesskey_IEEE = accesskey
        for req in range(1, max_pages, 1):
            request = requests.get("https://ieeexploreapi.ieee.org/api/v1/search/articles?query=(" + c_filter + ")&max_records=200&apikey=" + accesskey_IEEE)
            df_tmp = pd.DataFrame(json.loads(request.content)['articles'])
            lst.append(df_tmp)

    if 'ScienceDirect' in n_api:
        accesskey_SD = accesskey
        for req in range(1, max_pages, 1):
            request = requests.get("http://api.elsevier.com/content/search/scopus?query=(" + str(c_filter) + ")&show=100&apiKey=" + accesskey_SD)
            df_tmp = pd.DataFrame(dict(json.loads(request.content)['search-results'])['entry'])
            lst.append(df_tmp)

    return pd.concat(lst, ignore_index=True)

In [78]:
config = load_config('config.yaml')        
sch_str = config['search_string'][0]
accesskey_IEEE = 'efugu53u622asc77hu7h6hbq'
max_page = 10
df_ieee = get_data('IEEE', accesskey_IEEE, max_page ,sch_str)

# tratando colunas de issn
df_ieee['isbn'] = rmscaract(df_ieee.isbn)
df_ieee['issn'] = rmscaract(df_ieee.issn)

# tratamento da coluna author dicionario para string
df_ieee['authors'] = df_ieee['authors'].apply(lambda x : [a['full_name'] for a in x['authors']])
df_ieee['authors'] = df_ieee['authors'].map(lambda x : ', '.join(x))

# tratamento da coluna keywords dicionario para string (cria uma chave para as publicações sem o campo preenchido)
df_ieee['keywords'] = df_ieee['index_terms'].map(lambda x : x.get('ieee_terms', {'terms': []})['terms'])
df_ieee['keywords'] = df_ieee['keywords'].map(lambda x : ', '.join(x))
df_ieee['eissn'] = ''

# filtrando colunas
cols = ['doi', 'title', 'publisher', 'authors','keywords', 'content_type', 'issn', 'eissn', 'isbn', 'abstract', 'html_url',  'publication_title',  'publication_year']
df_ieee = df_ieee[cols]

# df_ieee.count()

In [80]:
accesskey_SD = 'ef8a7260c27897693d0fd7394a559726'
sch_str = config['search_string'][0]
max_page = 20
df_api_sd = get_data('ScienceDirect', accesskey_SD, max_page, sch_str)

# tratamento das colunas
df_api_sd['prism:issn'] = rmscaract(df_api_sd['prism:issn'])
df_api_sd['affiliation'] = df_api_sd['affiliation'].apply(lambda x : x[0]['affilname'])
df_api_sd['prism:coverDate'] = pd.DatetimeIndex(df_api_sd['prism:coverDate']).year
df_api_sd['keywords'] = ''
df_api_sd['abstract'] = ''
df_api_sd['isbn'] = ''

# filtrando colunas e renomeando colunas
cols = ['prism:doi', 'dc:title', 'affiliation', 'dc:creator', 'keywords', 'subtypeDescription', 'prism:issn', 'prism:eIssn', 'isbn', 'abstract', 'prism:url', 'prism:publicationName',  'prism:coverDate']
cols_rename = ['doi',      'title',    'publisher',   'authors', 'keywords', 'content_type',       'issn',       'eissn', 'isbn', 'abstract', 'html_url',  'publication_title',      'publication_year']

df_api_sd = df_api_sd[cols]
df_api_sd.columns = cols_rename
# df_api_sd.count()

In [110]:
df_api_union = pd.concat([df_ieee ,df_api_sd])
df_api_union
# df_api_union.count()


,doi,title,publisher,authors,keywords,content_type,issn,eissn,isbn,abstract,html_url,publication_title,publication_year
0,10.1109/CVPR.2016.90,Deep Residual Learning for Image Recognition,IEEE,"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun","Training, Degradation, Complexity theory, Imag...",Conferences,10636919,,9781467388528,Deeper neural networks are more difficult to t...,https://ieeexplore.ieee.org/document/7780459/,2016 IEEE Conference on Computer Vision and Pa...,2016
1,10.1109/TAC.1974.1100705,A new look at the statistical model identifica...,IEEE,H. Akaike,"Testing, Maximum likelihood estimation, Time s...",Journals,23343303,,,The history of the development of statistical ...,https://ieeexplore.ieee.org/document/1100705/,IEEE Transactions on Automatic Control,1974
2,10.1109/ICNN.1995.488968,Particle swarm optimization,IEEE,"J. Kennedy, R. Eberhart","Particle swarm optimization, Birds, Educationa...",Conferences,,,0780327683,A concept for the optimization of nonlinear fu...,https://ieeexplore.ieee.org/document/488968/,Proceedings of ICNN'95 - International Confere...,1995
3,10.1109/TIP.2003.819861,Image quality assessment: from error visibilit...,IEEE,"Zhou Wang, A.C. Bovik, H.R. Sheikh, E.P. Simon...","Image quality, Humans, Transform coding, Visua...",Journals,19410042,,,Objective methods for assessing perceptual ima...,https://ieeexplore.ieee.org/document/1284395/,IEEE Transactions on Image Processing,2004
4,10.1109/4235.996017,A fast and elitist multiobjective genetic algo...,IEEE,"K. Deb, A. Pratap, S. Agarwal, T. Meyarivan","Genetic algorithms, Sorting, Computational com...",Journals,19410026,,,Multi-objective evolutionary algorithms (MOEAs...,https://ieeexplore.ieee.org/document/996017/,IEEE Transactions on Evolutionary Computation,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
470,10.1016/j.omega.2022.102802,Store brand introduction in a dual-channel sup...,Nanjing University of Finance and EcoNomics,Xiao Y.,,Article,03050483,NaN,,,https://api.elsevier.com/content/abstract/scop...,Omega (United Kingdom),2023
471,10.1016/j.fm.2022.104167,Ethanol-lactate transition of Lachancea thermo...,Amsterdam Institute of Molecular and Life Scie...,Battjes J.,,Article,07400020,10959998,,,https://api.elsevier.com/content/abstract/scop...,Food Microbiology,2023
472,10.1016/j.fm.2022.104162,Determination and quantification of microbial ...,Quadram Institute Bioscience,Bloomfield S.J.,,Article,07400020,10959998,,,https://api.elsevier.com/content/abstract/scop...,Food Microbiology,2023
473,10.1016/j.rcim.2022.102468,Multiple source partial knowledge transfer for...,Nanjing Tech University,Liu X.,,Article,07365845,NaN,,,https://api.elsevier.com/content/abstract/scop...,Robotics and Computer-Integrated Manufacturing,2023


In [112]:
cols = ['issn', 'isbn', 'JournalTitle', 'publisher', 'title', 'doi', 'authors', 'keywords', 'abstract', 'publication_year', 'content_type', 'jcr_value', 'scimago_value']

df_join_api_csv = df_api_union.merge(dfjournalrank, left_on = 'issn', right_on = 'issnkey', how = 'left' )
df_join_api_csv = df_join_api_csv[cols]
# df_join_api_csv.count()

## Escrevendo os dados no banco de dados

In [90]:
#conexao com o banco

from sqlalchemy import create_engine

engine = create_engine('sqlite:///Bd_Publications.db', echo=False)
engine = engine.execution_options(autocommit=True)

In [113]:
engine.execute('''drop table tb_Publications''')

In [114]:
#criação da tabela
engine.execute('''CREATE TABLE tb_Publications (
        doi varchar(500) null,
        title varchar(500) null,
        JournalTitle varchar(500) null,
        publisher varchar(500) null,
        authors varchar(500) null,
        keywords varchar(500) null,
        issn varchar(50) null,
        eissn varchar(50) null,
        isbn varchar(50) null,
        abstract varchar(5000) null,
        html_url varchar(500) null,	
        content_type varchar(500) null,	
        publication_title varchar(500) null,	
        publication_year int null,
        jcr_value varchar(500) null,
        scimago_value varchar(500) null
    )''')

In [115]:
#setando a coluna que será o index
def insert_values_db(df_input, indx_col, tb_name):
    df=df_input.set_index(indx_col)

    #inserção tabela
    df.to_sql(tb_name, con=engine, if_exists='append')

insert_values_db(df_join_api_csv, 'doi', 'tb_Publications')



In [116]:
#leitura da tabela para validar se os dados foram escritos corretamente
field = 'publication_year'
value = '2016'

query = "SELECT * FROM tb_Publications where " + field + '=' + value + ";"
df_table = pd.read_sql_query(query, engine)
df_table.to_json(orient="index")

'{"0":{"doi":"10.1109\\/CVPR.2016.90","title":"Deep Residual Learning for Image Recognition","JournalTitle":"proceedings of the ieee computer society conference on computer vision and pattern recognition","publisher":"IEEE","authors":"Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun","keywords":"Training, Degradation, Complexity theory, Image recognition, Neural networks, Visualization, Image segmentation","issn":"10636919","eissn":null,"isbn":"9781467388528","abstract":"Deeper neural networks are more difficult to train. We present a residual learning framework to ease the training of networks that are substantially deeper than those used previously. We explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. We provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. On the ImageNet dataset we ev